In [1]:
from imutils.video import VideoStream
import imutils
import time
import cv2
import os

In [2]:
detectorPaths = {
	"face": "haarcascade_frontalface_default.xml",
	"eyes": "haarcascade_eye.xml",
	"smile": "haarcascade_smile.xml",
}

In [3]:
detect = cv2.CascadeClassifier()
detectors = {}

In [6]:
detectors

{'face': < cv2.CascadeClassifier 0x7f62b31733f0>,
 'eyes': < cv2.CascadeClassifier 0x7f62c843bcf0>,
 'smile': < cv2.CascadeClassifier 0x7f62b31b20b0>}

In [5]:
for name, path in detectorPaths.items():
    paths = os.path.sep.join([os.path.abspath(os.getcwd()), path])
    detectors[name] = cv2.CascadeClassifier(paths)

In [11]:
vs = VideoStream(src=0).start()
time.sleep(2)

[ WARN:0@105.025] global /io/opencv/modules/videoio/src/cap_v4l.cpp (902) open VIDEOIO(V4L2:/dev/video0): can't open camera by index


In [12]:
while True:
    frame = vs.read()
    
    # frame = imutils.resize(frame, width=500)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faceRects = detectors['face'].detectMultiScale(
        gray, scaleFactor=1.05, minNeighbors=5, minSize=(30, 30),
		flags=cv2.CASCADE_SCALE_IMAGE
    )

    for (fX, fY, fW, fH) in faceRects:
        faceROI = gray[fX:fX + fH, fY:fY + fW]
        eyeRects = detectors['eyes'].detectMultiScale(
            faceROI, scaleFactor=1.1, minNeighbors=10,
            minSize=(15, 15), flags=cv2.CASCADE_SCALE_IMAGE
        )

        smileRects = detectors['smile'].detectMultiScale(
            faceROI, scaleFactor=1.1, minNeighbors=10,
            minSize=(15, 15), flags=cv2.CASCADE_SCALE_IMAGE
        )

        for (eX, eY, eW, eH) in eyeRects:
            ptA = (fX + eX, fY + eY)
            ptB = (fX + eX + eW, fY + eY + eH)
            cv2.rectangle(frame, ptA, ptB, (0, 0, 255), 2)
        
        for (sX, sY, sW, sH) in smileRects:
            ptA = (sX + sX, sY + sY)
            ptB = (sX + sX + sW, sY + sY + sH)
            cv2.rectangle(frame, ptA, ptB, (255, 0, 0), 2)

        cv2.rectangle(frame, (fX, fY), (fX+fW, fY+fH), (0, 255, 0), 2)
    
    cv2.imshow('Frame', frame)
    key = cv2.waitKey(1) & 0xFF

    if key == ord('q'):
        break
cv2.destroyAllWindows()
vs.stop()

error: OpenCV(4.6.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
